In [2]:
import pandas as pd
import numpy as np
import nltk
import pickle
from nltk import word_tokenize, RegexpTokenizer,PunktSentenceTokenizer, sent_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#nltk.download('stopwords')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [80]:
def prepro(query):
    stop_words = stopwords.words('english')
    lemmatiser = WordNetLemmatizer()
    word_stemmer = PorterStemmer()
    count = 0
    tokens = gensim.utils.simple_preprocess(str(query), deacc=True)
    count += len(tokens)
    cleaned = [word for word in tokens if word not in stop_words]
    lemmatized = [lemmatiser.lemmatize(word_stemmer.stem(word)) for word in cleaned]
    return lemmatized

In [81]:
def normalise(query):
    tokens = gensim.utils.simple_preprocess(str(query), deacc=True)
    return tokens

In [7]:
all_info = pd.read_csv('graph_data_final.csv')

In [8]:
all_info.head()

,id,author,title,publication_date,publication_type,abstract,language,journal,organisation_unit_code,organisation,gender,department,department_leitzahl
0,108069.0,"Schützeichel, Rainer",Die Perspektive des Städtebauers. Ein Blick au...,2016,Book Chapter,NaN,de,Grundlagen,NaN,NaN,NaN,NaN,NaN
1,127413.0,"Schützeichel, Rainer",Einleitung. Wegmarken einer Theorie des archit...,2016,Book Chapter,NaN,de,NaN,NaN,NaN,NaN,NaN,NaN
2,127410.0,"Schützeichel, Rainer",Tradition as a Means of Modernisation: The Cru...,2016,Book Chapter,NaN,en,NaN,NaN,NaN,NaN,NaN,NaN
3,28599.0,"Schützeichel, Rainer",Architettura contestuale. Difesa di una proget...,2010,Book Chapter,NaN,it,NaN,NaN,NaN,NaN,NaN,NaN
4,319574.0,"Schützeichel, Rainer",Die Stadt und das Haus,2010,Book Chapter,NaN,en||de,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
def create_list(query_type):
    dep_org_prof = []
    for query in list(all_info[query_type].unique()):
        dep_org_prof.append(normalise(query))
    return dep_org_prof

In [93]:
author_name = create_list('author')
department = create_list('department') #Does not work - German
organisation = create_list('organisation')[1:] #First is nan


In [134]:
organisation

[['wagner', 'roy'],
 ['hagner', 'michael'],
 ['steinfeld', 'aldo'],
 ['dep', 'of', 'materials'],
 ['loffler', 'jorg'],
 ['institute', 'of', 'behavioral', 'sciences'],
 ['wingert', 'lutz'],
 ['clarysse', 'bart'],
 ['riener', 'robert'],
 ['institut'],
 ['for', 'landscape', 'architecture', 'former'],
 ['institute', 'for', 'technology', 'in', 'architecture'],
 ['cederman', 'lars', 'erik'],
 ['cis'],
 ['seminar', 'for', 'applied', 'mathematics'],
 ['corman', 'francesco'],
 ['inst', 'molecular', 'biology', 'and', 'biophysics'],
 ['stadler', 'tanja'],
 ['institute', 'of', 'fluid', 'dynamics'],
 ['dep', 'of', 'civil', 'env', 'and', 'geomatic', 'eng'],
 ['institute', 'of', 'molecular', 'health', 'sciences'],
 ['laboratory', 'of', 'organic', 'chemistry'],
 ['center', 'for', 'climate', 'systems', 'modeling'],
 ['inst', 'atmospheric', 'and', 'climate', 'science'],
 ['institute', 'of', 'history'],
 ['egger', 'peter'],
 ['institute', 'of', 'cartography', 'geoinformation'],
 ['vice', 'rector', 'for',

### Query Matching - Still need to do partial Matching

In [183]:
import re
def query_match(query):
    norm_query = normalise(query)
    matching_author = [v for v in author_name if set(norm_query) == set(v)]
    matching_org = [v for v in organisation if set(norm_query) == set(v)]
    if matching_author == [] and matching_org == []:
        print('not in author list')
        print('not an organisation')
    elif matching_author!=[]:
        print('--------Neo4J query: Author--------')
        print('MATCH (d:Department) - [:WORKS_IN] - (p:Person)')
        print('WHERE p.name = ', matching_author)
        print('RETURN d')
    elif matching_org != []:
        print('--------Neo4J query: Organisation--------')
        print('MATCH (d:Department) - [:WORKS_IN] - (p:Person)')
        print('WHERE p.name = ', matching_org)
        print('RETURN d')

In [186]:
query ='Andreas Krause is Professor at ETH'
norm_query = normalise(query)
pattern = re.compile("|".join(norm_query))
print(pattern)

re.compile('andreas|krause|is|professor|at|eth')


In [198]:
[sublist for sublist in author_name for item in sublist if pattern.match(item)]

[['trigo', 'isabel'],
 ['atmanspacher', 'harald'],
 ['andreasson', 'ingmar'],
 ['kabdaslı', 'isık'],
 ['diekmann', 'andreas'],
 ['sennheiser', 'andreas'],
 ['wenger', 'andreas'],
 ['horni', 'andreas'],
 ['tonnesmann', 'andreas'],
 ['schallhorn', 'andreas'],
 ['huber', 'andreas'],
 ['papritz', 'andreas', 'jurg'],
 ['bauder', 'andreas'],
 ['frei', 'andreas'],
 ['ulbig', 'andreas'],
 ['fischlin', 'andreas'],
 ['niang', 'diop', 'isabelle'],
 ['andreasen', 'lise'],
 ['hierlemann', 'andreas'],
 ['mansuy', 'isabelle'],
 ['biedermann', 'andreas'],
 ['stemmer', 'andreas'],
 ['ernst', 'andreas'],
 ['denk', 'andreas'],
 ['voigt', 'andreas'],
 ['brunschweiger', 'andreas'],
 ['mueller', 'andreas'],
 ['krause', 'andreas'],
 ['krause', 'andreas'],
 ['kispert', 'andreas'],
 ['bernasconi', 'andreas'],
 ['ruin', 'isabelle'],
 ['kunz', 'andreas'],
 ['schellenberger', 'andreas'],
 ['andreasen', 'rasmus'],
 ['isler', 'monika'],
 ['wigert', 'isabelle'],
 ['attinger'],
 ['schneider', 'andreas'],
 ['ohmura', 

In [143]:
query = 'Institute of Pharmaceutical Sciences'

In [185]:
query_match(query)

TypeError: expected string or bytes-like object

In [100]:
norm_query = normalise(query)

In [101]:
matching_author = [v for v in author_name if set(query) == set(v)]

In [103]:
matching_author

[]

In [104]:
if matching_author == []:
    print('empty')

empty


### Dictionary Method

In [9]:
def create_dic(query_type):
    dep_org_prof = all_info[query_type].unique()
    dep_org_prof = dep_org_prof[1:] # removed first as nan
    dic_out = {i: True for i in dep_org_prof}
    return dic_out

In [10]:
departments = create_dic('department')
organisations = create_dic('organisation')
professor = create_dic('author')

In [78]:
departments

{'Geistes-, Sozial- und Staatswissenschaften': True,
 'Maschinenbau und Verfahrenstechnik': True,
 'Gesundheitswissenschaften und Technologie': True,
 'Architektur': True,
 'Mathematik': True,
 'Bau, Umwelt und Geomatik': True,
 'Biosysteme': True,
 'Biologie': True,
 'Umweltsystemwissenschaften': True,
 'Management, Technologie und Ökonomie': True,
 'Physik': True,
 'Chemie und Angewandte Biowissenschaften': True,
 'Informationstechnologie und Elektrotechnik': True,
 'Informatik': True,
 'Materialwissenschaft': True,
 'Erdwissenschaften': True}